In [ ]:
import json

f = open('parameters.json')

data = json.load(f)

includes = data['include']
excludes = data['exclude']

if len(includes) > 0:
    print("there are includes")
elif len(excludes) > 0:
    print("there are excludes")
else:
    print("There are no parameters given")

In [ ]:
import requests

s = requests.Session()
s.headers.update({"platform":"pc", "language":"en"})

all_items = s.get("https://api.warframe.market/v1/items").json()
items = []


for index, possible_item in enumerate(all_items["payload"]["items"]):

    if possible_item["url_name"] == "baro_void_signal_(key)":
        print("test")
    if(any(substring in possible_item["url_name"] for substring in excludes)):
        print(possible_item["url_name"])
        continue
    else:
        items.append(possible_item["url_name"])
            
print(len(all_items["payload"]["items"]))
print(len(items))

#for i in items:
#    print(i)


In [ ]:
import statistics
import time
import csv

# write out results to sellPrices.csv
f = open('allOrderPrices.csv', 'w')
writer = csv.writer(f)
headers = ["url_name", "sell_mean", "sell_median", "buy_mean", "buy_median"]
writer.writerow(headers)

print("total items : " + str(len(all_items["payload"]["items"])))
print("reduced itemset : " + str(len(items)))

start_time = time.time();

for index, item in enumerate(items):

    url_name = item
    print(str(index) + "/" + str(len(items)) + " " + url_name)

    sellRequest = s.get("https://api.warframe.market/v1/items/" + url_name + "/orders").json()
    all_online_sell_prices = []
    all_online_buy_prices = []

    for sItem in sellRequest["payload"]["orders"]:
        user = sItem["user"]
        # ToDo : accept orders for players who have been offline, but for less than 12hr
        if user["status"] == "online" or user["status"] == "ingame":
            if(sItem["order_type"] == "sell"):
                all_online_sell_prices.append(sItem["platinum"])
            else:
                all_online_buy_prices.append(sItem["platinum"])

    # only add to table if item is available
    if len(all_online_sell_prices) > 0:
        sell_mean = statistics.mean(all_online_sell_prices)
        sell_median = statistics.median(all_online_sell_prices)
    if len(all_online_buy_prices) > 0:
        buy_mean = statistics.mean(all_online_buy_prices)
        buy_median = statistics.median(all_online_buy_prices)

    data = [url_name, sell_mean, sell_median, buy_mean, buy_median]
    writer.writerow(data)

    # Warframe.market API has a limit of 3 requests per second
    time.sleep(0.34)

f.close()
print("estimated time for total itemset: " + str(len(all_items["payload"]["items"]) / 3))
print("estimated time for reduced itemset: " + str(len(items)/ 3))
print("Total time elapsed: " + str(time.time() - start_time) + "s") 